In [1]:
from gnncloudmanufacturing.data import read_fatahi_dataset
from gnncloudmanufacturing.random_solution import random_solve
from gnncloudmanufacturing.validation import total_cost_from_graph, check_feasibility, total_cost_from_gamma
from gnncloudmanufacturing.utils import delta_from_gamma, graph_from_problem, gamma_from_target, delta_from_gamma
from gnncloudmanufacturing.graph_model import GNN, os_type, ss_type

import numpy as np
from tqdm.auto import trange, tqdm
from time import time
import pandas as pd
import torch

In [2]:
dataset = read_fatahi_dataset('../data/fatahi.xlsx')
for problem in dataset:
    print(f'Problem: {problem["name"]}')

  0%|          | 0/18 [00:00<?, ?it/s]

Problem: 5,10,10-1
Problem: 5,10,10-2
Problem: 5,10,10-3
Problem: 10,10,10-1
Problem: 10,10,10-2
Problem: 10,10,10-3
Problem: 5,10,20-1
Problem: 5,10,20-2
Problem: 5,10,20-3
Problem: 5,20,10-1
Problem: 5,20,10-2
Problem: 5,20,10-3
Problem: 5,20,20-1
Problem: 5,20,20-2
Problem: 5,20,20-3
Problem: 5,5,5-1
Problem: 5,5,5-2
Problem: 5,5,5-3


In [14]:
max_operations = 20
model = GNN.load_from_checkpoint(
    checkpoint_path="gnn_ver0.ckpt",
    ins_dim=1,
    ino_dim=max_operations,
    out_dim=16,
    n_layers=2,
    lr=0.001,
)

In [15]:
problem_name = []
total_cost = []
comp_time = []
for problem in tqdm(dataset):
    start = time()
    total = np.inf
    for i in range(5):
        graph = graph_from_problem(problem, max_operations=max_operations)
        graph.edata['feat'][os_type][:, 0] /= 10
        graph.edata['feat'][ss_type][:] /= 100
        pred = model.predict(graph)
        gamma = gamma_from_target(pred, graph, problem)
        delta = delta_from_gamma(problem, gamma)
        check_feasibility(gamma, delta, problem)
        _total = total_cost_from_gamma(problem, gamma, delta).item()
        if total > _total:
            total = _total
    total_cost.append(total)
    problem_name.append(problem['name'])
    comp_time.append(time() - start)

  0%|          | 0/18 [00:00<?, ?it/s]

In [16]:
pd.DataFrame({'problem_name': problem_name, 'total_cost': total_cost, 'comp_time': comp_time}).round(2)

,problem_name,total_cost,comp_time
0,"5,10,10-1",5086.08,0.12
1,"5,10,10-2",8814.99,0.09
2,"5,10,10-3",9807.67,0.08
3,"10,10,10-1",15853.54,0.12
4,"10,10,10-2",17028.23,0.12
5,"10,10,10-3",16682.86,0.13
6,"5,10,20-1",6091.27,0.19
7,"5,10,20-2",9019.25,0.20
8,"5,10,20-3",9452.16,0.25
9,"5,20,10-1",20492.35,0.13
